In [ ]:
import Structural_Perturbations_cifar as SP
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
"""Specifies batch size, number of classes in data and number of epochs, Data Augmentation is set to true"""

batch_size = 256
epochs = 100
data_augmentation = True
num_classes = 10


# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
# In[ ]:


# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
"""Model Architecture for ResNet"""

n = 3
depth = n*6+2
eval_batch_size = 2048

In [ ]:
"""Model Architecture as defined in https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py"""


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

    


In [ ]:
"""Data Augmentation. Please note that due to our nature of training, the augmentation is only limited to 
    structural pertubations not present in our work. Hence no rotation, no scaling etc"""
def model_train(model,x_train,y_train,callbacks):
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    x_train = x_train.astype('float32')/255
    datagen.fit(x_train)

    #     # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        use_multiprocessing=True,
        epochs=epochs,
        verbose=1,
        workers=40,
        callbacks=callbacks,
        steps_per_epoch=len(x_train) / batch_size)
    return model

In [ ]:
"""Defines architecture of the model. We use a ResNet architecture. Loss is categorical cross-entropy and optimiser 
    used is Adam with an epoch dependent learning rate scheduler"""
def new_model():
    global depth
    global input_shape
    model = resnet_v1(input_shape=input_shape, depth=depth)
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
# Prepare callbacks for model saving and for learning rate adjustment.

    lr_scheduler = LearningRateScheduler(lr_schedule)

    callbacks = [lr_scheduler]
    return model,callbacks
def evaluate(model,base,d,name,alpha,return_acc = 0):
    global epsilon
    print("Evaluating", alpha)
    res = []
    for i in alpha:
        data = np.copy(d)
        data = SP.Transform(name,data,i)
        labels = data[:,-1]
        labels = keras.utils.to_categorical(labels, 10)
        data = data[:,0:3072]
        data = data.reshape(-1,32,32,3)
        data = data.astype('float32')/255
        m1 = model.evaluate(data,labels,verbose = 0,batch_size=eval_batch_size)[1]
        res.append([i,m1])
    res = np.array(res)
    '''Check if the lowest model accuracy is below threshold. If yes find the largest change, calculate if accuracy
        at that point is below threshold and return the perturbation(i). If no, return out of range perturbation to quit'''
    
    if base - min(res[:, 1]) > epsilon:
        res = np.array(res)
        res = res[np.lexsort(np.fliplr(res).T)]
        df = derivative(res)
        for k in range(1, df[:, 1].shape[0]):
            i = df[:, 0][np.where(df[:, 1] == np.sort(df[:, 1])[-k])][0]
            if (base - res[:, 1][np.where(res[:, 0] == i)][0]) > epsilon:
#                 print("Accuracy at ", i, " = ",
#                       res[:, 1][np.where(res[:, 0] == i)])
                break

    
        """Plot the derivative values"""
#         plt.plot(df[:, 0], df[:, 1])
#         plt.axvline(i)
#         df = df[np.lexsort(np.fliplr(df).T)]
#         plt.show()
        return i, res
    else:
        return max(alpha)+1, res
    
"""Perturbs training dataset symmetrically and trains a given model on a perturbed dataset. Returns model"""
def train_model(name, d, alpha):
    data = np.copy(d)
    if name == 'Perspective':
        alpha.append(64-alpha[-1])
    elif name == "Scaling":
        alpha.append(2-alpha[-1])
    else:
        alpha.append(-alpha[-1])
    print("To be trained on ", alpha)
    for i in alpha:
        #Prevents duplicate natural data
        if i!=alpha[0]:
            new_data = SP.Transform(name,d,i)
            data = np.concatenate((data,new_data))
    model,callbacks = new_model()
    labels = data[:,-1]
    labels = keras.utils.to_categorical(labels, 10)
    data = data[:,0:3072].reshape(-1,32,32,3)
    model = model_train(model,data,labels,callbacks)
    return model

"""Loads data and stacks the labels as a column of data. This is required for multiprocessing."""
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    train_data = x_train.reshape(-1,3072)
    train_labels = y_train.reshape(-1,1)
    train = np.hstack((train_data,train_labels))
    eval_data = x_test.reshape(-1,3072)
    eval_labels = y_test.reshape(-1,1)
    input_shape = (32,32,3)
    test = np.hstack((eval_data,eval_labels))
    return (train,test,input_shape)

"""Return the derivative array"""
def derivative(res,width=1):
    res = np.array(res)
    res = res[np.lexsort(np.fliplr(res).T)]
    a = []
    for i in range(res.shape[0]-1):
        a.append([res[:,0][i],np.abs(res[:,1][i+1]-res[:,1][i])])
    return np.array(a)

In [ ]:
"""Loads the data and Vanilla model"""
train,test,input_shape = load_data()
model = load_model("Cifar-10-vanilla.h5")

In [ ]:
"""Finds the baseline accuracy of the vanilla model.
   Variables:
   
   res : Stores the results array of accurcy vs perturbation
   
   points : Initialize with initial (natural) pertubation.
            Scaling : 1
            Perspective : 28
            Else : 0
            Stores the points on which derivative was maximum and hence needs training.
            
   epsilon : Defines our threshold. 0.1 = 10% accuracy drop
   
   max_perturbation : Stores the maximum allowed perturbation
   
   name : Name of pertubration {Exposure, Scaling, Rotation, Translation, Shear, Perspective}
   
   x : step_length
   
   set_to_test : the array of pertubation to test on. 
                 Scaling ranges from 0 to a positive number.
                 Perspective ranges from 64-max to 28+max
                 Others range from -max to +max"""
base = model.evaluate(
    train[:, 0:3072].reshape(-1, 32, 32, 3) / 255,
    keras.utils.to_categorical(train[:, -1], 10),
    verbose=0,
    batch_size=eval_batch_size)[1]
res = []
points = []
epsilon = 0.1
print("Base Accuracy is ",base)

"""Make sure to change these three variables appropriately"""
name = "Rotation"
max_perturbation = 90
x = 30

set_to_test = []
if name == "Scaling":
    set_to_test = np.arange(2-max_perturbation,max_perturbation+x,x)
    points = [1]
elif name == "Perspective":
    set_to_test = np.arange(56-max_perturbation,max_perturbation+x,x)
    points = [28]
else:
    set_to_test = np.arange(-max_perturbation,max_perturbation+x,x)
    points = [0]

In [ ]:
"""The main loop. It continues til the model has >= accuracy than threshold on all pertubations in range"""

while(True):
    i,r = evaluate(model,base,train,name,set_to_test)
    if (i > max_perturbation):
        break
    res.append(r)
    points.append(i)
    model= train_model(name,train,points)
    print("New model created")
print("Model is robust")
points = np.array(points).round(decimals=2)
res = np.array(res)

In [ ]:
"""Test the robust model on test data over same pertubation range. Vertical lines show the training points"""
r = evaluate(model,base,test,name,set_to_test)[1]
plt.xlabel(name,fontsize = 18)
plt.ylim(0,1.1)
for i in points:
    plt.axvline(i,c='red',linewidth=0.5)
plt.ylabel("Accuracy",fontsize = 18)
plt.plot(r[:,0],r[:,1])
plt.show()

In [ ]:
"""Save the model, results array and array of points"""
model.save("CIFAR10_" + name +".h5")
np.save("CIFAR10_" + name +"_res.npy",res)
np.save("CIFAR10_"+ name +"_points.npy",points)